<a href="https://colab.research.google.com/github/MpMan123/Hallucination_Detection-Legendary-and-Mighty-Capybara/blob/main/notebooks/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%cd /content/
!git clone https://github.com/MpMan123/Hallucination_Detection-Legendary-and-Mighty-Capybara

/content
Cloning into 'Hallucination_Detection-Legendary-and-Mighty-Capybara'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 78 (delta 18), reused 40 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 6.10 MiB | 12.99 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [9]:
!ls /content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed/

evaluate.csv  train.csv


In [ ]:
import os
!ls
!ls /content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/config/

with open("/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/config/config_bert_base_multilingual.json", "r") as file:
  config = json.load(file)
print(config["model_name"])
print(config["batch_size"])
print(config["learning_rate"])
print(config["output_dir"])
print(config["log_dir"])


baseline.ipynb
config_bert_base_multilingual.json  config_model_unknown.json
bert-base-multilingual-cased
16
2e-05
results/checkpoints
results/logs/


In [51]:
!pip install evaluate
!pip install -U transformers

In [11]:
%cd /content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/
!ls
!git pull


/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src
config	models	training
Already up to date.


In [35]:
!git branch
!ls /content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/models

* main
base_model.py  __init__.py


In [40]:
import os

print(os.listdir("/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src"))
print(os.listdir("/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/models"))


['config', 'models', 'training']
['__init__.py', 'base_model.py']


In [42]:
import sys
import os

PROJECT_PATH = "/content/Hallucination_Detection-Legendary-and-Mighty-Capybara"
SRC_PATH = os.path.join(PROJECT_PATH, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

print(sys.path)


['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/tmp/tmpkuwhbns5', '/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src']


In [66]:
from transformers import (
                TrainingArguments,
                AutoModelForSequenceClassification,
                AutoTokenizer,
                Trainer,
                DataCollatorWithPadding)
import transformers
print(transformers.__version__)
import os
import json
import evaluate
from datasets import load_dataset

def load_model(model_name: str):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=3
    )
    return model

# Loading configuration
with open("/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/src/config/config_bert_base_multilingual.json", "r") as file:
  config = json.load(file)
MODEL_NAME = config["model_name"]
BATCH_SIZE = config["batch_size"]
LEARNING_RATE = config["learning_rate"]
NUM_EPOCHS = config["num_epochs"]
OUTPUT_DIR = config["output_dir"]
LOG_DIR = config["log_dir"]

# Load model
model = load_model(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load dataset
dataset = load_dataset("csv", data_files={
    "train":"/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed/train.csv",
    "evaluate":"/content/Hallucination_Detection-Legendary-and-Mighty-Capybara/data/processed/evaluate.csv"
})

label2id = {
    "extrinsic": 0,
    "intrinsic": 1,
    "hallucination": 2
}

def preprocess(examples):
    text = [c + " " + p + " " + r for c, p, r in zip(examples["context"], examples["prompt"], examples["response"])]
    return {
        **tokenizer(text, truncation=True, padding="max_length", max_length=256),
        "labels": [label2id[l] for l in examples["label"]]   # 👈 ép cả batch
    }

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.remove_columns(["id", "context", "prompt", "response"])
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format("torch")

# Metric functions
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def calculate_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    acc = accuracy.compute(predictions=preds, references=p.label_ids)
    f1_score = f1.compute(predictions=preds, references=p.label_ids, average="macro")
    return {"accuracy": acc["accuracy"], "f1_macro": f1_score["f1"]}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training args
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",          # vừa eval vừa save theo epoch
    save_strategy="epoch",
    save_total_limit=2,                   # chỉ giữ 2 checkpoint gần nhất
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    logging_dir=LOG_DIR,
    logging_steps=50,
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["evaluate"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=calculate_metrics
)
print(os.listdir(OUTPUT_DIR))
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

if __name__ == "__main__":
    trainer.train()
    trainer.save_model(os.path.join(OUTPUT_DIR, "final_model"))
    tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "final_model"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4.56.1


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

ValueError: invalid literal for int() with base 10: 'extrinsic'